In [1]:
# Import pandas
import pandas as pd

# Import bokeh
from bokeh.io import curdoc
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, Slider, Select, Button
from bokeh.layouts import column, row
from bokeh.palettes import Spectral6

In [2]:
# Import data
data = pd.read_csv("gapminder_tidy.csv", header=0, index_col="Year")
print(data.head())

          Country  fertility    life  population  child_mortality     gdp  \
Year                                                                        
1964  Afghanistan      7.671  33.639  10474903.0            339.7  1182.0   
1965  Afghanistan      7.671  34.152  10697983.0            334.1  1182.0   
1966  Afghanistan      7.671  34.662  10927724.0            328.7  1168.0   
1967  Afghanistan      7.671  35.170  11163656.0            323.3  1173.0   
1968  Afghanistan      7.671  35.674  11411022.0            318.1  1187.0   

          region  
Year              
1964  South Asia  
1965  South Asia  
1966  South Asia  
1967  South Asia  
1968  South Asia  


In [3]:
# Make the base plot


# Define initial plot axes
x = 'fertility'
xmin, xmax = min(data[x]), max(data[x])

y = 'life'
ymin, ymax = min(data[y]), max(data[y])

year = min(data.index)


# Define labels to make axis labels updated
label= {
    'fertility' : 'Fertility (children per woman)',
    'life' : 'Life Expectancy (years)',
    'child_mortality' : 'Child mortality rate',
    'gdp': 'Country GDP'
}


# Define bokeh data source
source = ColumnDataSource(data={
    'x'       : data.loc[year, x],
    'y'       : data.loc[year, y],
    'country' : data.loc[year, 'Country'],
    'pop'     : data.loc[year, 'population'],
    'pop_size': data.loc[year, 'population'] / 20000000 + 2,
    'region'  : data.loc[year, 'region']
})


# Make a color mapper
regions_list = data['region'].unique().tolist()
color_mapper = CategoricalColorMapper(factors=regions_list, palette=Spectral6)


# Create the figure
plot = figure(title='Gapminder data, plotting {} vs {} for year {}'.format(label[y], label[x], year), plot_height=700, plot_width=1000,
              x_range=(xmin, xmax), y_range=(ymin, ymax))

plot.circle(x='x', y='y', fill_alpha=0.6, source=source,
            color=dict(field='region', transform=color_mapper), size='pop_size', legend_group='region')

plot.legend.location = 'top_right'

plot.xaxis.axis_label = label[x]

plot.yaxis.axis_label = label[y]

In [4]:
# Create a HoverTool
hover = HoverTool(tooltips=[
    ('Country', '@country'),
    ('Region', '@region'),
    ('Population', '@pop{0,0}'),
    (label[x], '@x{0,0.00}'),
    (label[y], '@y{0,0.00}')
])

plot.add_tools(hover)

In [5]:
# Define the callback function for interacting with the widgets
def update_plot(attr, old, new):
    #Use global hover to update HoverTool    
    global hover
    
    #Get value of current widgets
    year = slider.value
    x = x_select.value
    y = y_select.value
    
    # Label plot axes
    plot.xaxis.axis_label = label[x]
    plot.yaxis.axis_label = label[y]
    
    # Set new_data
    new_data = {
        'x'       : data.loc[year, x],
        'y'       : data.loc[year, y],
        'country' : data.loc[year, 'Country'],
        'pop'     : data.loc[year, 'population'],
        'pop_size': data.loc[year, 'population'] / 20000000 + 2,
        'region'  : data.loc[year, 'region']
    }
    
    # Assign new_data to source.data
    source.data = new_data

    # Set the range of all axes
    plot.x_range.start = min(data[x])
    plot.x_range.end = max(data[x])
    plot.y_range.start = min(data[y])
    plot.y_range.end = max(data[y])

    # Add title to plot
    plot.title.text = 'Gapminder data, plotting {} vs {} for year {}'.format(label[y], label[x], year)
    
    # Updating the hover tools
    plot.tools.remove(hover)
    
    hover = HoverTool(tooltips=[
        ('Country', '@country'),
        ('Region', '@region'),
        ('Population', '@pop{0,0}'),
        (label[x], '@x{0,0.00}'),
        (label[y], '@y{0,0.00}')
    ])

    plot.add_tools(hover)


In [6]:
# Create a year slider
slider = Slider(start=min(data.index), end=max(data.index), step=1, value=min(data.index), title='Year')

# Attach the callback to slider
slider.on_change('value', update_plot)

# Create the list of the dropdown menus
dropdown = ['fertility', 'life', 'child_mortality', 'gdp']

# Create a dropdown Select widget for the x data
x_select = Select(
    options=dropdown,
    value=dropdown[0],
    title='x-axis data'
)

# Attach the update_plot callback to x dropdown
x_select.on_change('value', update_plot)

# Create a dropdown Select widget for the y data
y_select = Select(
    options=dropdown,
    value=dropdown[1],
    title='y-axis data'
)

# Attach the update_plot callback to y dropdown
y_select.on_change('value', update_plot)

In [7]:
# Create play-pause button

#define the callback function to animate the slider when clicking the button
def animate_update():
    year = slider.value + 1
    if year > max(data.index):
        year = min(data.index)
    slider.value = year

#create global variable for callback
callback_animate = None

#define the callback function for clicking the button
def animate():
    global callback_animate
    if button.label == '► Play':
        button.label = '❚❚ Pause'
        callback_animate = curdoc().add_periodic_callback(animate_update, 200)
    else:
        button.label = '► Play'
        curdoc().remove_periodic_callback(callback_animate)

#create the button and set on-click callback
button = Button(label='► Play', width=60)
button.on_click(animate)

In [8]:
# Create layout and add to current document
layout = row(column(slider, button, x_select, y_select), plot)
curdoc().add_root(layout)
curdoc().title = 'Gapminder'

In [9]:
# Run on Bokeh server with bash code
!bokeh serve --show gapminder_bokehserver.ipynb

2022-02-24 23:39:53,800 Starting Bokeh server version 2.3.2 (running on Tornado 6.1)
2022-02-24 23:39:53,804 User authentication hooks NOT provided (default user enabled)
2022-02-24 23:39:53,807 Bokeh app running at: http://localhost:5006/gapminder_bokehserver
2022-02-24 23:39:53,807 Starting Bokeh server with process id: 49860
          Country  fertility    life  ...  child_mortality     gdp      region
Year                                  ...                                     
1964  Afghanistan      7.671  33.639  ...            339.7  1182.0  South Asia
1965  Afghanistan      7.671  34.152  ...            334.1  1182.0  South Asia
1966  Afghanistan      7.671  34.662  ...            328.7  1168.0  South Asia
1967  Afghanistan      7.671  35.170  ...            323.3  1173.0  South Asia
1968  Afghanistan      7.671  35.674  ...            318.1  1187.0  South Asia

[5 rows x 7 columns]
2022-02-24 23:39:54,172 Error running application handler <bokeh.application.handlers.notebook.